# Introducción al manejo de datos geográficos

# Clase 2

## Parte 2. APIs de enriquecimiento geográfico
### Geolocalizando fachadas históricas en la Ciudad de Buenos Aires...

In [1]:
# importamos las librerías con las que vamos a trabajar 
import pandas as pd
import numpy as np

## Servicio de Normalización de Datos Geográficos (MM)

En nuestra primer clase vimos qué es un geocodificador y cómo se puede obtener las coordenadas geográficas de un punto específico a partir de la API de Google. Ahora vamos a ampliar un poco el espectro e introduciremos otra herramienta del estilo que desarrolló el Gobierno Nacional. Esta es la [API del Servicio de Normalización de Datos Geográficos de Argentina](https://datosgobar.github.io/georef-ar-api/), una herramienta muy útil para normalizar entidades territoriales, enriquecerlas u obtener información como sus coordenadas.

Esta herramienta nos será de mucha utilidad para el problema que necesitamos abordar: `obtener la latitud y longitud de un punto a partir de un string de direcciones físicas`.

In [2]:
# Primero instanciemos nuestro dataframe y construyamos nuestro string de direcciones
fachadas = pd.read_csv('../data/fachadas.csv')

In [3]:
# Como la altura es un valor numérico,
fachadas.calle_altura.dtype

dtype('float64')

In [4]:
# vamos a reemplazar los NaN por cero
fachadas.calle_altura.fillna(0, inplace=True)

In [5]:
# y ver cuántos son. Como es bajo el número vamos a suponer que los faltantes corresponden a la altura 0.
len(fachadas.loc[fachadas.calle_altura == 0])

14

In [6]:
# Y ahora convertimos las alturas en integer,
fachadas.calle_altura = fachadas.calle_altura.map(int)

In [7]:
# para luego guardarlas como string.
fachadas.calle_altura = fachadas.calle_altura.astype(str)

In [8]:
# Armamos un único string
fachadas['direccion'] = fachadas['calle_nombre'] + ' ' + fachadas['calle_altura']

Ahora que ya tenemos nuestro string listo, veamos un primer ejemplo para enriquecer nuestras entidades geogŕaficas revisando algunos de los [ejemplos en python](https://datosgobar.github.io/georef-ar-api/python-usage/).

Lo que vamos a hacer es tomar el caso de normalización de varias entidades que se lleva a cabo para consultas provinciales. Por medio de la función `similar_bulk` se construirá un diccionario con el listado de direcciones. Se utilizará el parámetro `endpoint`, con el nombre de lo que será la llave con la que se accederá a una lista de diccionarios. Cada uno de esos diccionarios contará con la key `dirección` que dará acceso al `str` que construimos más arriba con el nombre de la dirección. Es decir, la lista de direcciones que pasamos como segundo parámetro. 

También vamos a agregar en el cuerpo de la función una información adicional: el `departamento` al que pertenece nuestra entidad. Así es como se guarda la data que será consultada en el endpoint a través del request. Las keys dirección y departamento son las que darán acceso a la base de datos de donde se consumirá la información adicional que nos traeremos de vuelta. 

Para esto nos fijamos cómo se construye la API BASE para direcciones en los [ejemplos de uso](https://datosgobar.github.io/georef-ar-api/quick-start/). Así, podemos ver que el formato de la consulta es el siguiente: `https://apis.datos.gob.ar/georef/api/direcciones?departamento=merlo&direccion=Florida al 2950`. En nuestro caso, como estamos en la Ciudad de Buenos Aires nuestro departamento serán las Comunas. Al tener 15 posibilidades, podría ser que debiéramos haber completado con el `str` de las `15 Comunas`. Para lo cual, deberíamos haber incorporado un parámetro adicional en la función `get_similar_bulk`. Algo que hiciera masomenos lo siguiente:

```
def get_similar_bulk(endpoint, nombres, departamento):
    (...)
    return parsed_results
```

Pero eso hubiese llevado a la tarea de introducir la consulta en un loop que iterara sobre el rango 1 a 15 y testeara las quince posibilidades diferentes para cada dirección - dado que por la información con la que cuenta nuestra base, tampoco sabemos a qué comuna pertenece cada una de ellas-. Algo verdaderamente costoso en consumo de memoria. Pero por suerte la API resuelve muy bien este caso de uso. Con sólo utilizar la palabra `Comuna` para el parámetro `departamento` que se consumirá en el endpoint será suficiente para traer la consulta sin error.  

In [9]:
# Importemos los módulos necesarios para el uso de la API
import requests

In [10]:
# Construimos un string con la url base que vamos a usar para construir las consultas
API_BASE_URL = "https://apis.datos.gob.ar/georef/api/"

In [11]:
# Creamos la función
def get_similar_bulk(endpoint, nombres):
    '''
    Normaliza una lista de nombres de alguna de 
    las entidades geográficas.
    ...
    Argumentos:
        endpoint(str): String de texto.
        nombres (iter): Objeto iterable (e.g: list, Series)
    Devuelve:
        list: diccionarios con el resultado de la consulta 
    '''

    # realiza consulta a la API
    data = {
        endpoint: [
            {"direccion": nombre,
             "departamento":'Comuna',
             "max":5} for nombre in nombres
    ]}
    url = API_BASE_URL + endpoint
    results = requests.post(
        url, json=data, headers={"Content-Type": "application/json"}
    ).json()
    
    try:
        parsed_results = [
            single_result[endpoint][0] if single_result[endpoint] else {'Sin dato'}
            for single_result in results["resultados"]
        ]
        print('Consulta realizada: %r resultados obtenidos' % len(parsed_results))

    except:
        print('Excediste el limite de consultas')
        parsed_results = {'Sin dato'}

    return parsed_results

Si comparamos la función con la del ejemplo provisto en la web de la API se puede identificar que agregamos un `try`/`except` al final. Antes de usar este tipo de servicios, es recomendable leer las [condiciones de uso](https://datosgobar.github.io/georef-ar-api/terms/). En el caso de esta API, las [consultas por lote](https://datosgobar.github.io/georef-ar-api/bulk/) tienen límites máximos. Veamos qué pasa si no los respetamos...  

In [12]:
# Este es el total de direcciones que deberíamos consultar
len(fachadas.direccion)

26489

In [13]:
nombres = fachadas.direccion

In [14]:
# Podemos ver que en algún momento de la consulta, la API dejó de responder y pasamos al bloque except.
direcciones = get_similar_bulk("direcciones", nombres)

Excediste el limite de consultas


Esto es porque la cantidad de consultas en una misma petición no debe superar las `1000`. Nosotros definimos 5 como `max`, con lo cual si ahora vamos por 999 no deberían tener ningún problema en hacer la consulta.

In [15]:
import numpy as np

In [16]:
def consultas(totales):
    '''
    Aplica get_similar_bulk sobre
    la serie de direcciones de un df
    ...
    Argumentos:
        totales (int): límite superior del intervalo.
    Devuelve:
        list: diccionarios con el resultado de cada consulta 
    '''
    intervalos = np.arange(0, totales, 999).tolist()
    intervalos.append(totales)
    
    listado = []
    for i in range(len(intervalos)):
        try:
            print("Iterando de %r a %r casos" % (intervalos[i], intervalos[i+1]))
            I = intervalos[i]
            F = intervalos[i+1]
        except:
            print("Consulta terminada")
            break

        nombres = fachadas.direccion.iloc[I:F]
        nombres.fillna('Sin dato', inplace=True)
        direcciones = get_similar_bulk("direcciones", nombres)
        listado.extend(direcciones)
        
    return listado

In [17]:
# Aplicamos get_similar_bulk sobre todo el listado de direcciones de nuestro df
para_enriquecer = consultas(len(fachadas))

Iterando de 0 a 999 casos
Consulta realizada: 999 resultados obtenidos
Iterando de 999 a 1998 casos
Consulta realizada: 999 resultados obtenidos
Iterando de 1998 a 2997 casos
Consulta realizada: 999 resultados obtenidos
Iterando de 2997 a 3996 casos
Consulta realizada: 999 resultados obtenidos
Iterando de 3996 a 4995 casos
Consulta realizada: 999 resultados obtenidos
Iterando de 4995 a 5994 casos
Consulta realizada: 999 resultados obtenidos
Iterando de 5994 a 6993 casos
Consulta realizada: 999 resultados obtenidos
Iterando de 6993 a 7992 casos
Consulta realizada: 999 resultados obtenidos
Iterando de 7992 a 8991 casos
Consulta realizada: 999 resultados obtenidos
Iterando de 8991 a 9990 casos
Consulta realizada: 999 resultados obtenidos
Iterando de 9990 a 10989 casos
Consulta realizada: 999 resultados obtenidos
Iterando de 10989 a 11988 casos
Consulta realizada: 999 resultados obtenidos
Iterando de 11988 a 12987 casos
Consulta realizada: 999 resultados obtenidos
Iterando de 12987 a 13986

In [18]:
# Tenemos la misma cantidad de casos en nuestra consulta
len(para_enriquecer)

26489

In [19]:
# y en nuestro dataframe original
len(fachadas)

26489

In [20]:
# Y estas son las keys que tenemos en cada consulta. Ponemos la primera sólo para verlas.
para_enriquecer[0].keys()

dict_keys(['altura', 'calle', 'calle_cruce_1', 'calle_cruce_2', 'departamento', 'localidad_censal', 'nomenclatura', 'piso', 'provincia', 'ubicacion'])

In [21]:
def enriquece(consultas, atributo):
    '''
    Accede al item de cada key de 
    obtenido en la conuslta.
    ...
    Argumentos:
        consultas (list): lista de diccionarios.
        atributo (str): nombre de la key
    Devuelve:
        list: string con el atributo consultado 
    '''
    resultado = []
    for i in range(len(consultas)):
        if type(consultas[i]) is dict:
            if atributo in consultas[i].keys():
                resultado.append(consultas[i][atributo])
        else:
            resultado.append('Sin resultado')
    return resultado

In [22]:
# Nos traemos la nomenclatura y vemos que seguimos teniendo la misma cantidad de casos. Es un buen signo!
len(enriquece(para_enriquecer, 'nomenclatura'))

26489

In [23]:
# Ahora sumamos esta info a nuestro dataframe
fachadas['nomenclatura'] = enriquece(para_enriquecer, 'nomenclatura')

In [24]:
fachadas.head()

,seccion,manzana,parcela,partida,calle_nombre,calle_altura,vencimiento_certificado,direccion,nomenclatura
0,21,2,9,429255.0,"ALVAREZ, JULIAN",2531,2019-03-12,"ALVAREZ, JULIAN 2531","JULIAN ALVAREZ 2531, Comuna 14, Ciudad Autónom..."
1,79,89,22,260164.0,"LAMARCA, EMILIO",1014,2019-03-30,"LAMARCA, EMILIO 1014","EMILIO LAMARCA 1014, Comuna 11, Ciudad Autónom..."
2,17,14,27,173288.0,SARMIENTO,4579,2019-05-03,SARMIENTO 4579,"SARMIENTO 4579, Comuna 5, Ciudad Autónoma de B..."
3,42,16,1,140186.0,DIRECTORIO AV.,1706,2019-06-10,DIRECTORIO AV. 1706,"AV DIRECTORIO 1706, Comuna 7, Ciudad Autónoma ..."
4,11,65,6,442215.0,ALVEAR AV.,1974,2019-06-16,ALVEAR AV. 1974,"AV ALVEAR 1974, Comuna 2, Ciudad Autónoma de B..."


Ahora tenemos una columna de nomenclatura con toda la dirección completa. Cómo sería enriquecer nuestro dataframe con el resto de los atributos? Por ejemplo, con la información del departamento...

In [25]:
# Como se puede ver, algunos casos son listas de diccionarios donde cada key es un potencial atributo...
pd.Series(enriquece(para_enriquecer, 'departamento')).head(3)

0    {'id': '02098', 'nombre': 'Comuna 14'}
1    {'id': '02077', 'nombre': 'Comuna 11'}
2     {'id': '02035', 'nombre': 'Comuna 5'}
dtype: object

Para estos casos, vamos a construir una funcion que nos permita ir recolectando los atributos de interés. Por ejemplo...

In [26]:
def devuelve_subatributo(atributo, subatributo):
    '''
    Accede a los items de las listadas almacenadas
    dentro del diccionario.
    ...
    Argumentos:
        atributo(str): string de texto con el nombre de la key.
        subatributo (list): lista de string con el nombre de
                            de los subatributos.
    Devuelve:
        item: elemento de cada key ordenada en la lista. 
    '''
    atributos = enriquece(para_enriquecer, atributo)
    return [atributos[i][subatributo] if type(atributos[i]) 
            is dict else 'Sin dato' for i in range(len(atributos))]

In [27]:
# Construimos los parametros de la función
targets = {'departamento':['id','nombre'],
           'ubicacion':['lat','lon'],
           'localidad_censal':['id','nombre']}

In [28]:
# y la aplcamos en un for loop.
for k,v in targets.items():
    print(k, v[0], v[1])
    fachadas[v[0]],fachadas[v[1]] = devuelve_subatributo(k,v[0]), devuelve_subatributo(k,v[1])

departamento id nombre
ubicacion lat lon
localidad_censal id nombre


**Ahora, qué problema ven con esto? Se guardan las columnas como nosotros queremos o hay algún efecto no deseado?** 

**Podrían identificar cuál es el problema?** 

In [29]:
# Solución
for k,v in targets.items():
    print(k, v[0], v[1])
    if k == 'departamento':
        fachadas[v[0]+'dto'],fachadas[v[1]+'dto'] = devuelve_subatributo(k,v[0]), devuelve_subatributo(k,v[1])
    else:
        fachadas[v[0]],fachadas[v[1]] = devuelve_subatributo(k,v[0]), devuelve_subatributo(k,v[1])     

departamento id nombre
ubicacion lat lon
localidad_censal id nombre


Efetivamente, como dos de nuestros atributos tenían el mismo nombre, el último pisaba el primero y por eso sólo veíamos las columnas de id y nombre del último caso iterado. Veamos ahora nuestro dataframe enriquecido...

In [30]:
fachadas.head(2)

,seccion,manzana,parcela,partida,calle_nombre,calle_altura,vencimiento_certificado,direccion,nomenclatura,id,nombre,lat,lon,iddto,nombredto
0,21,2,9,429255.0,"ALVAREZ, JULIAN",2531,2019-03-12,"ALVAREZ, JULIAN 2531","JULIAN ALVAREZ 2531, Comuna 14, Ciudad Autónom...",02000010,Ciudad Autónoma de Buenos Aires,-34.5862,-58.4138,02098,Comuna 14
1,79,89,22,260164.0,"LAMARCA, EMILIO",1014,2019-03-30,"LAMARCA, EMILIO 1014","EMILIO LAMARCA 1014, Comuna 11, Ciudad Autónom...",02000010,Ciudad Autónoma de Buenos Aires,-34.6222,-58.4827,02077,Comuna 11


In [31]:
# Exportemoslo para no tener que correr todo de nuevo
#fachadas.to_csv('fachadas_sndg.csv', index=False)

## Normalizador AMBA (USIG-GCBA)

Otro servicio similar es el [normalizador de direcciones](https://pypi.org/project/usig-normalizador-amba/) del Gobierno de la Ciudad de Buenos Aires. Antes de comenzar a utilizarlo, vamos a hacer algunas aclaraciones. Revisando el [release history](https://pypi.org/project/usig-normalizador-amba/#history), constatamos que la [versión más reciente](https://github.com/usig/normalizador-amba) debería estar en `1.3.0`. Sin embargo, una versión más reciente (`2.1.2`) se encuentra disponible en [sitios oficiales](http://servicios.usig.buenosaires.gob.ar/normalizar). Aparentemente esta aún no se encuentra disponible en `pip`. Por lo tanto, iremos recorriendo algunos ejemplos de la última release de Git y revisando el instructivo disponible en el último hipervínculo compartido más arriba. Prosigamos...

In [32]:
# Instalemos desde pip si es que aún no lo hiciste
#!pip install usig-normalizador-amba

In [33]:
# Importamos el normalizador
from usig_normalizador_amba import NormalizadorAMBA

Algo interesante de esta API (que no vimos en la anterior) es que permite normalizar una dirección parseando strings de texto y eligiendo el distrito objetivo (esto siempre dentro del Área Metropolitana de Buenos Aires). En la función que armamos a continuación vemos dos formas diferentes de instanciar el normalizador. Introduciendo un `Boolean` parameter vamos a elegir cuál de los dos usar. Ambos devuelven la misma información pero nos sirven para resolver situaciones diferentes. Si nuestro input ya se encuentra formateado como dirección, es decir, no se encuentra con formatos del estilo `calle 1, esquina calle 2` - por mencionar sólo un ejemplo - utilizaremos el normalizador simplemente. Mientras que en el segundo caso lo instanciaremos como `buscarDireccion`.  

In [34]:
def normaliza(direccion, parsea_texto):
    '''
    Normaliza una dirección geográficas.
    ...
    Argumentos:
        dirección (str): String de texto.
        parsea_texto (bool): Buscar dirección en texto crudo.
    Devuelve:
        dict: nombre, altura y coords de la dirección. 
    '''
    nd = NormalizadorAMBA(include_list=['caba'])
    string = u'{}'.format(direccion)
    
    try:
        if parsea_texto:
            res = nd.buscarDireccion(string)
            for r in res:
                n = res[0][0]['direcciones'][0].calle.nombre
                a = res[0][0]['direcciones'][0].altura
                c = res[0][0]['direcciones'][0].coordenadas
                
            return {'nombre': n, 
                    'altura': a,
                    'coord': c}
        
        else:
            res = nd.normalizar(string)
        
            for r in res:
                n = r.calle.nombre
                a = r.altura
                c = r.coordenadas

            return {'nombre': n, 
                    'altura': a,
                    'coord': c}
    
    except Exception as e:
        print('error')

In [35]:
# Apliquemos esta función sobre las dos primeras direcciones de nuestro dataframe
fachadas.direccion[0:2]

0    ALVAREZ, JULIAN 2531
1    LAMARCA, EMILIO 1014
Name: direccion, dtype: object

In [36]:
# Como las direcciones están en una serie de pandas podemos usar la función anónima lambda
fachadas.direccion[0:2].apply(lambda x: normaliza(x, False))

0    {'nombre': 'ALVAREZ, JULIAN', 'altura': 2531, ...
1    {'nombre': 'LAMARCA, EMILIO', 'altura': 1014, ...
Name: direccion, dtype: object

In [37]:
# Probemos cómo hubiese sido si nuestras direcciones se encontraban en un formato más crudo
pd.Series(['Julián Álvarez al 2531', 'emilio lamarca 1014 (y galicia)']).apply(lambda x: normaliza(x,'Texto'))

0    {'nombre': 'ALVAREZ, JULIAN', 'altura': 2531, ...
1    {'nombre': 'LAMARCA, EMILIO', 'altura': 1014, ...
dtype: object

Aparentemente, en nuestro dataframe las direcciones ya se encontraban normalizadas. Esto lo podemos constatar viendo cómo se normalizan los casos en formato más crudo. Ahora bien, parece haber un problema. Y este es que en ninguna de las dos oportunidades nos duevle coordenadas:

In [38]:
#1
print(fachadas.direccion[0:2].apply(lambda x: normaliza(x, False))[0]['coord'])

None


In [39]:
#2
print(pd.Series(['Julián Álvarez al 2531', 
                 'emilio lamarca 1014 (y galicia)']).apply(lambda x: normaliza(x,'Texto'))[0]['coord'])

None


Antes de continuar, evaluemos un poco la performance del normalizador. Qué tiempo nos llevaría correrlo sobre toda nuestra serie de direcciones?

In [40]:
import time
# calculamos tiempo de ejecución
inicio = time.time()
partes = fachadas.direccion[0:100].apply(lambda x: normaliza(x, 'Texto'))
fin = time.time()

In [41]:
print('Tiempo de consulta para 1000 casos: %r segundos' % (fin-inicio))

Tiempo de consulta para 1000 casos: 85.5688693523407 segundos


In [42]:
# Eso quiere decir que haberlo aplicado a toda nuestra serie hubiese llevado...
tiempo_total = (((fin - inicio)*26000)/100)/60
print('Tiempo de consulta para la totalidad de casos: %r minutos' % tiempo_total)

Tiempo de consulta para la totalidad de casos: 370.798433860143 minutos


### Modularizando el normalizador...

Dado que hasta ahora no hemos conseguido las coordenadas geográficas con el normalizador, vamos a combinar algunos pasos que encapsularemos en una función única. Como se ejemplifica en esta [consulta](http://servicios.usig.buenosaires.gob.ar/normalizar/?direccion=juli%C3%A1n%20alvarez%202351,%20caba), la url `http://servicios.usig.buenosaires.gob.ar/normalizar` puede ser completada con una `/direccion=(...)` que devuelve la metadata de la misma. 

Lo que vamos a hacer a continuación es disponer de la librería `urllib` para para traernos ese resultado. Para ello, mostraremos dos caminos alternativos pero con el mismo punto de llegada. Uno largo y otro corto. En el primero veremos cómo trabajar con recursos en su mayoría nativos de python. En el segundo, apelando a métodos específicamente diseñados para lidiar con la situación que debemos sortear.

### EL CAMINO LARGO

El problema que tenemos que resolver es básicamente la adaptación del formato de nuestros strings de direcciones. Como necesitamos que estos nombres se ubiquen en el contexto de una url, es importante que los mismos sean legibles en código [ASCII](https://www.w3schools.com/charsets/ref_html_ascii.asp) para que la consulta se efectúe correctamente

<img align="left" width="500" height="125" src="../imagenes/camino_largo.jpeg" style="float: left; padding: 0 15px">
Lo que especificamente necesitamos hacer es enviar un request a una página web utilizando un URL. Las URLs pueden enviarse a Internet solamente con un formato ASCII válido. Dado que nuestros string de direcciones van a contener caracteres que están por fuera de este encoding, debemos garantizar que los caracteres no seguros sean transformados en el contexto mismo de la URL. 

Casos como espacios entre palabras o tildes deberán ser reemplazados por su equivalente en ASCII. Tal es así que en esta seccion mostraremos cómo construir la dirección que debemos incluir en la consulta de una manera más pythonesca, apelando a métodos que son aplicables al string de texto que iremos modificando para darle un encoding compatible a la url. Realizaremos algunas [transormaciones](https://www.w3schools.com/tags/ref_urlencode.ASP) que harán que la dirección que enviemos sea legible en el request.

Comencemos entonces por crear el grupo de funciones con las que modificaremos nuestra serie de direcciones. Lo que haremos por medio de estas es, en primer lugar, separar el nombre de la dirección de su altura y completar los espacios entre strings con el código ASCII que los reemplaza.

In [43]:
def separa_partes(string):
    '''
    Divide caracteres alfa numéricos dentro
    de un string.
    ...
    Argumentos:
        string(str): String de texto.
    Devuelve:
        dict: calle y altura. 
    '''
    calle, altura = '', ''

    for i in string:
        if i.isdigit():
            altura+=str(i)
        else:
            calle+=str(i)
            
    return {'nombre': calle,
            'altura': altura}        

In [44]:
# Vemos el resultado de nuestra función
separa_partes(fachadas.direccion[1])

{'nombre': 'LAMARCA, EMILIO ', 'altura': '1014'}

In [45]:
def unifica_direccion(calle, altura):
    '''
    Unifica caracteres alfanuméricos
    reemplazando espacios por %20
    ...
    Argumentos:
        calle(str): string de letras.
        altura(str): string de numeros
    Devuelve:
        str: direccion unificada. 
    '''
    nombre_url = []
    for i in calle:
        if i != ' ':
            nombre_url.append(i)
        else:
            nombre_url.append('%20')
    nombre_url.extend('%20')
    
    nombre_completo = ''.join([''+i for i in nombre_url])
    
    return nombre_completo+str(altura)

In [46]:
# Instanciamos el resultado de nuestra función, 
resultado = unifica_direccion(separa_partes(fachadas.direccion[0])['nombre'],
                              separa_partes(fachadas.direccion[0])['altura'])

In [47]:
# y vemos qué nos devuelve. 
resultado

'ALVAREZ,%20JULIAN%20%202531'

Como se puede apreciar en el ejemplo de [consulta](http://servicios.usig.buenosaires.gob.ar/normalizar/?direccion=juli%C3%A1n%20alvarez%202351,%20caba) que compartimos previamente, la dirección se encuentra codificada en un formato legible para la consulta que haremos a la url. Lo que hicimos acá es reemplazar los espacios por un `%20` para que nuestro string se acomode al encoding de ASCII. Ahora, vamos a proceder a hacer el request en sí.

In [48]:
# Importamos el módulo request de urllib,
import urllib.request

In [49]:
# Instanciamos el string de la consulta sobre el que iremos cambiando el valor de la dirección
url = 'http://servicios.usig.buenosaires.gob.ar/normalizar/?direccion={},%20caba'.format(resultado)

In [50]:
# Por ejemplo, con lo que guardamos en 'resultado'
contents = urllib.request.urlopen(url).read()

In [51]:
# esto es lo que nos devuelve
contents

b'{"direccionesNormalizadas": [{"altura": 2531, "cod_calle": 1057, "cod_calle_cruce": null, "cod_partido": "caba", "coordenadas": {"srid": 4326, "x": "-58.413591", "y": "-34.586111"}, "direccion": "ALVAREZ, JULIAN 2531, CABA", "nombre_calle": "ALVAREZ, JULIAN", "nombre_calle_cruce": "", "nombre_localidad": "CABA", "nombre_partido": "CABA", "tipo": "calle_altura"}]}'

In [52]:
# Ahora convertimos el btype bytes en un diccionario para recolectar nuestros resultados
import json

In [53]:
# para lo que cargamos nuestro resultado como json
coords = json.loads(contents.decode('utf-8'))

In [54]:
coords['direccionesNormalizadas'][0]['coordenadas']

{'srid': 4326, 'x': '-58.413591', 'y': '-34.586111'}

Ahora sí, modularizamos estos pasos en una funcion que nos devuelva las coordenadas. Lo que primero hará es adaptar el string de entrada al formato que usaremos en el url. Luego, lo introducirá en la consulta y, si esta fuese errónea pasará nuevamente el string por el normalizador. Este buscará una dirección en el texto que le pasamos para devolver otra normalizada que iniciará nuevamente el proceso de formateo para reintentar la consulta. 

In [55]:
def devuelve_coordenadas(direccion):
    '''
    Procesa una dirección y realiza una consulta 
    web al normalizador USIG.
    ...
    Argumentos:
        direccion(str): string de letras y numeros.
    Devuelve:
        dict: epsg, longitud y latitud de la direccion. 
    '''
    
    try:
        # Contruye formato compatible con urlopen
        tildes = {'á': 'a',
                  'é': 'e',
                  'í': 'i', 
                  'ó': 'o',
                  'ú': 'u'}

        for k in tildes.keys():
            if k in direccion:
                direccion = direccion.replace(k, tildes[k])
            else:
                pass

        formato_web = unifica_direccion(separa_partes(direccion)['nombre'],
                                        separa_partes(direccion)['altura'])

        url = 'http://servicios.usig.buenosaires.gob.ar/normalizar/?direccion={},%20caba'
        consulta = url.format(formato_web)

        # Consulta el servicio de georreferencimiento y carga los resultados
        print('Direccion ingresada: %s' % direccion)
        resultado = urllib.request.urlopen(consulta).read()
        carga_resultado = json.loads(resultado.decode('utf-8'))
        print('Direccion conultada: %s' % direccion)


        # Si la consulta falla, volvemos a pasar una direccion normalizada
        if 'errorMessage' in carga_resultado.keys():
            # Normalizamos la direccion para pasar un string legible
            normalizar = normaliza(direccion, 'Texto')
            direccion_normalizada = normalizar['nombre']+' '+str(normalizar['altura'])
            print('Consultando direccion: %s' % direccion_normalizada)
            nvo_formato_web = unifica_direccion(separa_partes(direccion_normalizada)['nombre'],
                                                separa_partes(direccion_normalizada)['altura'])

            nva_consulta = url.format(nvo_formato_web)
            nvo_resultado = urllib.request.urlopen(nva_consulta).read()
            nva_carga_resultado =json.loads(nvo_resultado.decode('utf-8'))
            info = nva_carga_resultado['direccionesNormalizadas'][0]['coordenadas']
        else:
            info = carga_resultado['direccionesNormalizadas'][0]['coordenadas']

        return info
    except:
        return 'Calle inexistente'

In [56]:
# Recordamos que fachadas.direccion[0] es
fachadas.direccion[0]

'ALVAREZ, JULIAN 2531'

In [57]:
# Probamos nuestra función
devuelve_coordenadas('ALVAREZ, JULIAN 2531')

Direccion ingresada: ALVAREZ, JULIAN 2531
Direccion conultada: ALVAREZ, JULIAN 2531


{'srid': 4326, 'x': '-58.413591', 'y': '-34.586111'}

In [58]:
# Y ahora vemos qué hubiese devuelto si la dirección estaba escrita de otra manera
devuelve_coordenadas('jUlián álVarez 2531 (esquina arenales)')

Direccion ingresada: jUlian alVarez 2531 (esquina arenales)
Direccion conultada: jUlian alVarez 2531 (esquina arenales)
Consultando direccion: ALVAREZ, JULIAN 2531


{'srid': 4326, 'x': '-58.413591', 'y': '-34.586111'}

In [59]:
# Vemos cuanto tarda en consultar mil casos
inicio = time.time()
coordenadas = fachadas.direccion[0:100].map(devuelve_coordenadas)
fin = time.time()

Direccion ingresada: ALVAREZ, JULIAN 2531
Direccion conultada: ALVAREZ, JULIAN 2531
Direccion ingresada: LAMARCA, EMILIO 1014
Direccion conultada: LAMARCA, EMILIO 1014
Direccion ingresada: SARMIENTO 4579
Direccion conultada: SARMIENTO 4579
Direccion ingresada: DIRECTORIO AV. 1706
Direccion conultada: DIRECTORIO AV. 1706
Direccion ingresada: ALVEAR AV. 1974
Direccion conultada: ALVEAR AV. 1974
Direccion ingresada: LAS HERAS GENERAL AV. 2062
Direccion conultada: LAS HERAS GENERAL AV. 2062
Direccion ingresada: NOGOYA 5138
Direccion conultada: NOGOYA 5138
Direccion ingresada: PAVON 2361
Direccion conultada: PAVON 2361
Direccion ingresada: PIEDRAS 534
Direccion conultada: PIEDRAS 534
Direccion ingresada: CHILE 577
Direccion conultada: CHILE 577
Direccion ingresada: ARGERICH 698
Direccion conultada: ARGERICH 698
Direccion ingresada: SAN MARTIN 1109
Direccion conultada: SAN MARTIN 1109
Direccion ingresada: RECONQUISTA 962
Direccion conultada: RECONQUISTA 962
Direccion ingresada: SUIPACHA 1183

In [60]:
tiempo_total = (((fin - inicio)*26000)/100)/60
print('Tiempo de consulta para la totalidad de casos: %r minutos' % tiempo_total)

Tiempo de consulta para la totalidad de casos: 58.51223556200663 minutos


### EL CAMINO CORTO 

<img align="right" width="525" height="125" src="../imagenes/camino_corto.jpeg" style="float: right; padding: 0 15px">

En lugar de adaptar manualmente los caracteres que están por fuera del encoding de la url, ahora vamos a apelar a una herramienta pensada específicamente para codificar una url. Esta es el módulo [parse de urllib](https://www.urlencoder.io/python/). Con el método `quote()` podemos acomodar cada uno de los caracteres de nuestras direcciones sin tener que estar atentos a cada uno de ellos específicamente. 

Esto nos permitirá ahorrar bastantes líneas de código y acortar considerablemente el cuerpo de nuestra función `devuelve_coordenadas`. Ya no buscaremos espacios o tildes, por ejemplo. Simplemente, pasaremos nuestro string por este método y obtendremos un resultado legible en ASCII encoding.  


In [61]:
# Importamos parse
import urllib.parse

# y utilizamos el método quote para obtener un string en formato ascii
urllib.parse.quote(fachadas.direccion[0]) 

'ALVAREZ%2C%20JULIAN%202531'

In [62]:
# Veamos cómo el método quote nos permite acortar la función
def devuelve_coordenadas(direccion):
    '''
    Procesa una dirección y realiza una consulta 
    web al normalizador USIG.
    ...
    Argumentos:
        direccion(str): string de letras y numeros.
    Devuelve:
        dict: epsg, longitud y latitud de la direccion. 
    '''
    
    try:
        # Contruye formato compatible con urlopen
        url = 'http://servicios.usig.buenosaires.gob.ar/normalizar/?direccion={},%20caba'
        direccion_ascii = urllib.parse.quote(direccion)
        consulta = url.format(direccion_ascii)

        # Consulta el servicio de georreferencimiento y carga los resultados
        print('Direccion ingresada: %s' % direccion)
        resultado = urllib.request.urlopen(consulta).read()
        carga_resultado = json.loads(resultado.decode('utf-8'))
        print('Direccion consultada: %s' % direccion)
        info = carga_resultado['direccionesNormalizadas'][0]['coordenadas']
        return info
    except:
        return 'Calle sin resultado'

In [63]:
# Ejecutemos la función y veamos cuánto tiempo lleva
inicio = time.time()
coordenadas = fachadas.direccion[0:100].map(devuelve_coordenadas)
fin = time.time()

Direccion ingresada: ALVAREZ, JULIAN 2531
Direccion consultada: ALVAREZ, JULIAN 2531
Direccion ingresada: LAMARCA, EMILIO 1014
Direccion consultada: LAMARCA, EMILIO 1014
Direccion ingresada: SARMIENTO 4579
Direccion consultada: SARMIENTO 4579
Direccion ingresada: DIRECTORIO AV. 1706
Direccion consultada: DIRECTORIO AV. 1706
Direccion ingresada: ALVEAR AV. 1974
Direccion consultada: ALVEAR AV. 1974
Direccion ingresada: LAS HERAS GENERAL AV. 2062
Direccion consultada: LAS HERAS GENERAL AV. 2062
Direccion ingresada: NOGOYA 5138
Direccion consultada: NOGOYA 5138
Direccion ingresada: PAVON 2361
Direccion consultada: PAVON 2361
Direccion ingresada: PIEDRAS 534
Direccion consultada: PIEDRAS 534
Direccion ingresada: CHILE 577
Direccion consultada: CHILE 577
Direccion ingresada: ARGERICH 698
Direccion consultada: ARGERICH 698
Direccion ingresada: SAN MARTIN 1109
Direccion consultada: SAN MARTIN 1109
Direccion ingresada: RECONQUISTA 962
Direccion consultada: RECONQUISTA 962
Direccion ingresada: 

In [64]:
tiempo_total = (((fin - inicio)*26000)/100)/60
print('Tiempo de consulta para la totalidad de casos: %r minutos' % tiempo_total)

Tiempo de consulta para la totalidad de casos: 92.64578898747762 minutos


Evaluando uno contra otro, no es que el camino corto nos permitirá ejecutar nuestra función de una manera más rápida. Las dos tardan aproximadamente lo mismo. Algo que era esperable, porque ambas deben hacer una consulta (lo único en lo que difieren es la manera en la que codifican el string). Cuestión que no es menor, ya que apelando al método `quote` no sólo conseguimos que nuestra función ahorre líneas de código. También nos asegura no estar sujeto a posibles excepciones (caracteres que no conocemos en nuestra serie de direcciones dada su extensión). Este método estandariza lo que haga falta de manera general sin tener que estar atento caracter por caracter, lo que indudablemente hace que nuestra función sea más consistente. 

In [65]:
# mapeamos nustra función a toda la serie de direcciones
coordenadas = fachadas.direccion.map(devuelve_coordenadas)

Direccion ingresada: ALVAREZ, JULIAN 2531
Direccion consultada: ALVAREZ, JULIAN 2531
Direccion ingresada: LAMARCA, EMILIO 1014
Direccion consultada: LAMARCA, EMILIO 1014
Direccion ingresada: SARMIENTO 4579
Direccion consultada: SARMIENTO 4579
Direccion ingresada: DIRECTORIO AV. 1706
Direccion consultada: DIRECTORIO AV. 1706
Direccion ingresada: ALVEAR AV. 1974
Direccion consultada: ALVEAR AV. 1974
Direccion ingresada: LAS HERAS GENERAL AV. 2062
Direccion consultada: LAS HERAS GENERAL AV. 2062
Direccion ingresada: NOGOYA 5138
Direccion consultada: NOGOYA 5138
Direccion ingresada: PAVON 2361
Direccion consultada: PAVON 2361
Direccion ingresada: PIEDRAS 534
Direccion consultada: PIEDRAS 534
Direccion ingresada: CHILE 577
Direccion consultada: CHILE 577
Direccion ingresada: ARGERICH 698
Direccion consultada: ARGERICH 698
Direccion ingresada: SAN MARTIN 1109
Direccion consultada: SAN MARTIN 1109
Direccion ingresada: RECONQUISTA 962
Direccion consultada: RECONQUISTA 962
Direccion ingresada: 

Direccion consultada: PEDERNERA 1030
Direccion ingresada: TRIUNVIRATO AV. 2883
Direccion consultada: TRIUNVIRATO AV. 2883
Direccion ingresada: GARCIA, MARTIN AV. 417
Direccion consultada: GARCIA, MARTIN AV. 417
Direccion ingresada: ROBERTSON 740
Direccion consultada: ROBERTSON 740
Direccion ingresada: SOLER 4263
Direccion consultada: SOLER 4263
Direccion ingresada: LAVALLE 416
Direccion consultada: LAVALLE 416
Direccion ingresada: MUÑIZ 469
Direccion consultada: MUÑIZ 469
Direccion ingresada: ROBERTSON 640
Direccion consultada: ROBERTSON 640
Direccion ingresada: MORETO 629
Direccion consultada: MORETO 629
Direccion ingresada: CAMPANA 2450
Direccion consultada: CAMPANA 2450
Direccion ingresada: HUMBOLDT 2160
Direccion consultada: HUMBOLDT 2160
Direccion ingresada: COCHABAMBA 373
Direccion consultada: COCHABAMBA 373
Direccion ingresada: DEL LIBERTADOR AV. 6037
Direccion consultada: DEL LIBERTADOR AV. 6037
Direccion ingresada: BONORINO, ESTEBAN, Cnel. AV. 349
Direccion consultada: BONORIN

Direccion consultada: BRASIL AV. 351
Direccion ingresada: AVELLANEDA AV. 2676
Direccion consultada: AVELLANEDA AV. 2676
Direccion ingresada: RIVADAVIA AV. 4953
Direccion consultada: RIVADAVIA AV. 4953
Direccion ingresada: PAVON 2144
Direccion consultada: PAVON 2144
Direccion ingresada: PARAGUAY 4156
Direccion consultada: PARAGUAY 4156
Direccion ingresada: BORGES, JORGE LUIS 2337
Direccion consultada: BORGES, JORGE LUIS 2337
Direccion ingresada: BEIRO, FRANCISCO AV. 4318
Direccion consultada: BEIRO, FRANCISCO AV. 4318
Direccion ingresada: GARAY, JUAN DE AV. 3033
Direccion consultada: GARAY, JUAN DE AV. 3033
Direccion ingresada: URUGUAY AV. 632
Direccion consultada: URUGUAY AV. 632
Direccion ingresada: PEÑA 2939
Direccion consultada: PEÑA 2939
Direccion ingresada: NEWBERY, JORGE 2441
Direccion consultada: NEWBERY, JORGE 2441
Direccion ingresada: CASTAÑON 956
Direccion consultada: CASTAÑON 956
Direccion ingresada: ROOSEVELT FRANKLIN D. 3674
Direccion consultada: ROOSEVELT FRANKLIN D. 3674

Direccion consultada: LACROZE, FEDERICO AV. 2137
Direccion ingresada: VILLAFAÑE, WENCESLAO 1420
Direccion consultada: VILLAFAÑE, WENCESLAO 1420
Direccion ingresada: OLLEROS 4159
Direccion consultada: OLLEROS 4159
Direccion ingresada: ARENALES 1289
Direccion consultada: ARENALES 1289
Direccion ingresada: RINCON 294
Direccion consultada: RINCON 294
Direccion ingresada: CHACABUCO 1305
Direccion consultada: CHACABUCO 1305
Direccion ingresada: ALBERDI, JUAN BAUTISTA AV. 4367
Direccion consultada: ALBERDI, JUAN BAUTISTA AV. 4367
Direccion ingresada: BALCARCE 783
Direccion consultada: BALCARCE 783
Direccion ingresada: DEL LIBERTADOR AV. 134
Direccion consultada: DEL LIBERTADOR AV. 134
Direccion ingresada: ACOYTE AV. 735
Direccion consultada: ACOYTE AV. 735
Direccion ingresada: CASEROS AV. 1952
Direccion consultada: CASEROS AV. 1952
Direccion ingresada: CAMPICHUELO 644
Direccion consultada: CAMPICHUELO 644
Direccion ingresada: MELIAN AV. 3823
Direccion consultada: MELIAN AV. 3823
Direccion ing

Direccion consultada: CASTRO BARROS 980
Direccion ingresada: PIEDRAS 1179
Direccion consultada: PIEDRAS 1179
Direccion ingresada: GUEMES 4147
Direccion consultada: GUEMES 4147
Direccion ingresada: DEL LIBERTADOR AV. 5346
Direccion consultada: DEL LIBERTADOR AV. 5346
Direccion ingresada: 15 DE NOVIEMBRE DE 1889 2554
Direccion consultada: 15 DE NOVIEMBRE DE 1889 2554
Direccion ingresada: CHARCAS 2975
Direccion consultada: CHARCAS 2975
Direccion ingresada: RIVADAVIA AV. 3729
Direccion consultada: RIVADAVIA AV. 3729
Direccion ingresada: ROSARIO 398
Direccion consultada: ROSARIO 398
Direccion ingresada: PARAGUAY 4263
Direccion consultada: PARAGUAY 4263
Direccion ingresada: ROSARIO 302
Direccion consultada: ROSARIO 302
Direccion ingresada: GUEMES 3334
Direccion consultada: GUEMES 3334
Direccion ingresada: SUPERI 1426
Direccion consultada: SUPERI 1426
Direccion ingresada: CIUDAD DE LA PAZ 2626
Direccion consultada: CIUDAD DE LA PAZ 2626
Direccion ingresada: SANTA FE AV. 4981
Direccion consult

...

Direccion consultada: DOBLAS 278
Direccion ingresada: MOLDES 1423
Direccion consultada: MOLDES 1423
Direccion ingresada: BELGRANO AV. 2630
Direccion consultada: BELGRANO AV. 2630
Direccion ingresada: PRINGLES 835
Direccion consultada: PRINGLES 835
Direccion ingresada: POTOSI 3824
Direccion consultada: POTOSI 3824
Direccion ingresada: HERNANDEZ, JOSE 1974
Direccion consultada: HERNANDEZ, JOSE 1974
Direccion ingresada: BONIFACIO, JOSE 1971
Direccion consultada: BONIFACIO, JOSE 1971
Direccion ingresada: MELINCUE 3175
Direccion consultada: MELINCUE 3175
Direccion ingresada: OLAVARRIA 547
Direccion consultada: OLAVARRIA 547
Direccion ingresada: MELINCUE 3172
Direccion consultada: MELINCUE 3172
Direccion ingresada: OLAVARRIA 571
Direccion consultada: OLAVARRIA 571
Direccion ingresada: DIAZ VELEZ AV. 4752
Direccion consultada: DIAZ VELEZ AV. 4752
Direccion ingresada: ZABALA 3436
Direccion consultada: ZABALA 3436
Direccion ingresada: PERON, JUAN DOMINGO, TTE. GENERAL 725
Direccion consultada: 

Direccion consultada: CHARCAS 2777
Direccion ingresada: ASAMBLEA AV. 819
Direccion consultada: ASAMBLEA AV. 819
Direccion ingresada: CHENAUT, INDALESIO, GRAL. AV. 1723
Direccion consultada: CHENAUT, INDALESIO, GRAL. AV. 1723
Direccion ingresada: SAN JUAN AV. 2278
Direccion consultada: SAN JUAN AV. 2278
Direccion ingresada: JUSTO, JUAN B. AV. 5071
Direccion consultada: JUSTO, JUAN B. AV. 5071
Direccion ingresada: ALEM, LEANDRO N. AV. 1134
Direccion consultada: ALEM, LEANDRO N. AV. 1134
Direccion ingresada: DEAN FUNES 1631
Direccion consultada: DEAN FUNES 1631
Direccion ingresada: BOEDO AV. 1707
Direccion consultada: BOEDO AV. 1707
Direccion ingresada: SOLER 3902
Direccion consultada: SOLER 3902
Direccion ingresada: OLLEROS AV. 1785
Direccion consultada: OLLEROS AV. 1785
Direccion ingresada: GALLARDO, ANGEL AV. 349
Direccion consultada: GALLARDO, ANGEL AV. 349
Direccion ingresada: GUAYAQUIL 348
Direccion consultada: GUAYAQUIL 348
Direccion ingresada: SUCRE, ANTONIO JOSE de, Mcal. 2355
Di

Direccion consultada: FORMOSA 228
Direccion ingresada: BELGRANO AV. 2211
Direccion consultada: BELGRANO AV. 2211
Direccion ingresada: MANZANARES 1766
Direccion consultada: MANZANARES 1766
Direccion ingresada: LOYOLA 102
Direccion consultada: LOYOLA 102
Direccion ingresada: BRAGADO 4910
Direccion consultada: BRAGADO 4910
Direccion ingresada: HUMAHUACA 4650
Direccion consultada: HUMAHUACA 4650
Direccion ingresada: GUATEMALA 4502
Direccion consultada: GUATEMALA 4502
Direccion ingresada: JURAMENTO 3002
Direccion consultada: JURAMENTO 3002
Direccion ingresada: PARAGUAY 4628
Direccion consultada: PARAGUAY 4628
Direccion ingresada: AMENABAR 3458
Direccion consultada: AMENABAR 3458
Direccion ingresada: LARRALDE, CRISOLOGO AV. 4023
Direccion consultada: LARRALDE, CRISOLOGO AV. 4023
Direccion ingresada: DIRECTORIO AV. 1398
Direccion consultada: DIRECTORIO AV. 1398
Direccion ingresada: MITRE, BARTOLOME 3564
Direccion consultada: MITRE, BARTOLOME 3564
Direccion ingresada: LA RIOJA 2047
Direccion c

Direccion consultada: GOROSTIAGA 2298
Direccion ingresada: SOLANO LOPEZ, F., MARISCAL 3376
Direccion consultada: SOLANO LOPEZ, F., MARISCAL 3376
Direccion ingresada: ARCE 430
Direccion consultada: ARCE 430
Direccion ingresada: PACHECO DE MELO, JOSE ANDRES 2927
Direccion consultada: PACHECO DE MELO, JOSE ANDRES 2927
Direccion ingresada: CIUDAD DE LA PAZ 3719
Direccion consultada: CIUDAD DE LA PAZ 3719
Direccion ingresada: VUELTA DE OBLIGADO 4473
Direccion consultada: VUELTA DE OBLIGADO 4473
Direccion ingresada: BOEDO 243
Direccion consultada: BOEDO 243
Direccion ingresada: ALSINA, ADOLFO 2256
Direccion consultada: ALSINA, ADOLFO 2256
Direccion ingresada: CONGRESO 1524
Direccion consultada: CONGRESO 1524
Direccion ingresada: TINOGASTA 2863
Direccion consultada: TINOGASTA 2863
Direccion ingresada: ARGERICH 4832
Direccion consultada: ARGERICH 4832
Direccion ingresada: FORMOSA 118
Direccion consultada: FORMOSA 118
Direccion ingresada: CASTRO BARROS AV. 314
Direccion consultada: CASTRO BARRO

Direccion consultada: VIAMONTE 2778
Direccion ingresada: GURRUCHAGA 1134
Direccion consultada: GURRUCHAGA 1134
Direccion ingresada: ROJAS 593
Direccion consultada: ROJAS 593
Direccion ingresada: SARMIENTO 543
Direccion consultada: SARMIENTO 543
Direccion ingresada: RABANAL, FRANCISCO, INTENDENTE AV. 2950
Direccion consultada: RABANAL, FRANCISCO, INTENDENTE AV. 2950
Direccion ingresada: MEDRANO AV. 836
Direccion consultada: MEDRANO AV. 836
Direccion ingresada: BULNES 1813
Direccion consultada: BULNES 1813
Direccion ingresada: FIGUEROA ALCORTA, PRES. AV. 3490
Direccion consultada: FIGUEROA ALCORTA, PRES. AV. 3490
Direccion ingresada: PASEO COLON AV. 50
Direccion consultada: PASEO COLON AV. 50
Direccion ingresada: UGARTE, MANUEL 1740
Direccion consultada: UGARTE, MANUEL 1740
Direccion ingresada: PARAGUAY 1639
Direccion consultada: PARAGUAY 1639
Direccion ingresada: CAPDEVILA, ARTURO 1686
Direccion consultada: CAPDEVILA, ARTURO 1686
Direccion ingresada: THORNE 333
Direccion consultada: THO

In [66]:
# Nos devolvió al menos algún valor para todos nuestros casos. La extensión es igual al de la serie de direcciones.
len(coordenadas)

26489

In [67]:
# Por cada row nos devuelve un diccionario que debemos transformar en columnas.
pd.Series(coordenadas)[0:3]

0    {'srid': 4326, 'x': '-58.413591', 'y': '-34.58...
1    {'srid': 4326, 'x': '-58.482847', 'y': '-34.62...
2                                  Calle sin resultado
Name: direccion, dtype: object

In [68]:
# Hacemos una copia de nuestro dataframe de fachadas
fachadas_ = fachadas.copy()

In [69]:
# Accedemos a la key srid de cada item de nuestra serie de direcciones con una lista por comprension
epsg = [coordenadas[i]['srid'] if coordenadas[i] != 'Calle sin resultado' else 
        'Sin dato' for i in range(len(coordenadas))]

In [70]:
# El resultado es el que esperabamos, tenemos la misma cantidad de casos.
len(epsg)

26489

In [71]:
# Hacemos lo mismo con la longitud
lon = [coordenadas[i]['x'] if coordenadas[i] != 'Calle sin resultado' else 
        'Sin dato' for i in range(len(coordenadas))]

In [72]:
len(lon)

26489

In [73]:
# y la latitud.
lat = [coordenadas[i]['y'] if coordenadas[i] != 'Calle sin resultado' else 
        'Sin dato' for i in range(len(coordenadas))]

In [74]:
# Y guardamos todo como series de nuestro df de fachadas
fachadas_['lat'], fachadas_['lon'], fachadas_['epsg'] = lat, lon, epsg

In [75]:
fachadas_.head()

,seccion,manzana,parcela,partida,calle_nombre,calle_altura,vencimiento_certificado,direccion,nomenclatura,id,nombre,lat,lon,iddto,nombredto,epsg
0,21,2,9,429255.0,"ALVAREZ, JULIAN",2531,2019-03-12,"ALVAREZ, JULIAN 2531","JULIAN ALVAREZ 2531, Comuna 14, Ciudad Autónom...",02000010,Ciudad Autónoma de Buenos Aires,-34.586111,-58.413591,02098,Comuna 14,4326
1,79,89,22,260164.0,"LAMARCA, EMILIO",1014,2019-03-30,"LAMARCA, EMILIO 1014","EMILIO LAMARCA 1014, Comuna 11, Ciudad Autónom...",02000010,Ciudad Autónoma de Buenos Aires,-34.622146,-58.482847,02077,Comuna 11,4326
2,17,14,27,173288.0,SARMIENTO,4579,2019-05-03,SARMIENTO 4579,"SARMIENTO 4579, Comuna 5, Ciudad Autónoma de B...",02000010,Ciudad Autónoma de Buenos Aires,Sin dato,Sin dato,02035,Comuna 5,Sin dato
3,42,16,1,140186.0,DIRECTORIO AV.,1706,2019-06-10,DIRECTORIO AV. 1706,"AV DIRECTORIO 1706, Comuna 7, Ciudad Autónoma ...",02000010,Ciudad Autónoma de Buenos Aires,-34.630757,-58.451690,02049,Comuna 7,4326
4,11,65,6,442215.0,ALVEAR AV.,1974,2019-06-16,ALVEAR AV. 1974,"AV ALVEAR 1974, Comuna 2, Ciudad Autónoma de B...",02000010,Ciudad Autónoma de Buenos Aires,-34.587221,-58.389945,02014,Comuna 2,4326


In [76]:
# Exportemoslo para no tener que correr todo de nuevo
#fachadas_.to_csv('data/fachadas_usig.csv', index=False)